In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gradio as gr
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [5]:
sns.set(style="whitegrid")

def visualize_and_predict(csv_file):
    df = pd.read_csv(csv_file.name)

    # --- Plot 1: Bar plot for average Sales by Ad Medium ---
    avg_sales = {
        'TV': df['TV'].mean(),
        'Radio': df['Radio'].mean(),
        'Newspaper': df['Newspaper'].mean()
    }

    plt.figure(figsize=(6, 4))
    plt.bar(avg_sales.keys(), avg_sales.values(), color=['blue', 'green', 'orange'])
    plt.title("Average Advertising Spend")
    plt.ylabel("Amount Spent")
    plot1 = plt.gcf()

    # --- Plot 2: Line chart for total ad spend vs Sales trend ---
    df['Total_Spend'] = df[['TV', 'Radio', 'Newspaper']].sum(axis=1)
    df_sorted = df.sort_values(by='Total_Spend')

    plt.figure(figsize=(6, 4))
    plt.plot(df_sorted['Total_Spend'], df_sorted['Sales'], marker='o', color='purple')
    plt.title("Sales vs Total Advertising Spend")
    plt.xlabel("Total Ad Spend")
    plt.ylabel("Sales")
    plot2 = plt.gcf()

    # --- Plot 3: Pie chart of Ad Spend Proportions ---
    spend_sum = df[['TV', 'Radio', 'Newspaper']].sum()
    plt.figure(figsize=(5, 5))
    plt.pie(spend_sum, labels=spend_sum.index, autopct='%1.1f%%', colors=['skyblue', 'lightgreen', 'lightcoral'])
    plt.title("Ad Spend Distribution")
    plot3 = plt.gcf()

    # --- Linear Regression & Prediction Plot ---
    X = df[['TV', 'Radio', 'Newspaper']]
    y = df['Sales']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)

    plt.figure(figsize=(6, 4))
    plt.plot(y_test.values, label='Actual Sales', marker='o')
    plt.plot(y_pred, label='Predicted Sales', marker='x')
    plt.title("Actual vs Predicted Sales")
    plt.xlabel("Sample")
    plt.ylabel("Sales")
    plt.legend()
    plot4 = plt.gcf()

    model_info = f"""🔍 Model Summary:
TV Coefficient: {model.coef_[0]:.2f}
Radio Coefficient: {model.coef_[1]:.2f}
Newspaper Coefficient: {model.coef_[2]:.2f}
Intercept: {model.intercept_:.2f}
R² Score: {r2:.2f}
"""

    return plot1, plot2, plot3, plot4, model_info

In [6]:
inputs = gr.File(label="📂 Upload CSV with 'TV', 'Radio', 'Newspaper', and 'Sales' columns")
outputs = [
    gr.Plot(label="📊 Average Advertising Spend (Bar Plot)"),
    gr.Plot(label="📈 Sales vs Total Spend (Line Chart)"),
    gr.Plot(label="🥧 Ad Spend Distribution (Pie Chart)"),
    gr.Plot(label="📉 Actual vs Predicted Sales (Line Chart)"),
    gr.Textbox(label="🧠 Model Information")
]

gr.Interface(
    fn=visualize_and_predict,
    inputs=inputs,
    outputs=outputs,
    title="📈 Advertising Analysis & Sales Prediction",
    description="Upload a CSV file with columns 'TV', 'Radio', 'Newspaper', and 'Sales'. The app shows user-friendly visualizations and predicts sales."
).launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
